# Capstone project Week 2
# Battle of the Stadiums!

# Importing Stadium and GDP data

In [1]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6dc6e76ff8b7428e9b620d2caaddd26d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='7kB_7QCeHaxzC7fQFXGHOdkXHp19IFf3SMjZIjET0bWV',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_6dc6e76ff8b7428e9b620d2caaddd26d.get_object(Bucket='capstoneproject-donotdelete-pr-ltxl82bdztwxva',Key='stadiums-with-GPS-coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Team,FDCOUK,City,Stadium,Capacity,Latitude,Longitude,Country
0,Arsenal,Arsenal,London,Emirates Stadium,60361,51.555000,-0.108611,England
1,Aston Villa,Aston Villa,Birmingham,Villa Park,42785,52.509167,-1.884722,England
2,Blackburn Rovers,Blackburn,Blackburn,Ewood Park,31154,53.728611,-2.489167,England
3,Bolton Wanderers,Bolton,Bolton,Reebok Stadium,28100,53.580556,-2.535556,England
4,Chelsea,Chelsea,London,Stamford Bridge,42449,51.481667,-0.191111,England


In [8]:

body = client_6dc6e76ff8b7428e9b620d2caaddd26d.get_object(Bucket='capstoneproject-donotdelete-pr-ltxl82bdztwxva',Key='GDP.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_0 = pd.read_excel(body)
df_data_0.head()


,Ranking,Economy,"US dollars, millions",Unnamed: 3
0,1,United States,2.04941e+07,NaN
1,2,China,1.36082e+07,NaN
2,3,Japan,4.97092e+06,NaN
3,4,Germany,3.99676e+06,NaN
4,5,United Kingdom,2.82521e+06,NaN


# Cleaning and merging data

In [10]:
df_data_0 = df_data_0.rename(columns={'Economy':'Country'})

In [15]:
df_data_0['Country'].replace('United Kingdom', 'England',inplace=True)

In [73]:
result = pd.merge(df_data_1, 
                 df_data_0,
                 on='Country', 
                 how='left')

In [74]:
result = result.drop('FDCOUK', 1)

In [75]:
result = result.drop('Unnamed: 3', 1)

In [22]:
result.head()

,Team,City,Stadium,Capacity,Latitude,Longitude,Country,Ranking,"US dollars, millions"
0,Arsenal,London,Emirates Stadium,60361,51.555000,-0.108611,England,5,2.82521e+06
1,Aston Villa,Birmingham,Villa Park,42785,52.509167,-1.884722,England,5,2.82521e+06
2,Blackburn Rovers,Blackburn,Ewood Park,31154,53.728611,-2.489167,England,5,2.82521e+06
3,Bolton Wanderers,Bolton,Reebok Stadium,28100,53.580556,-2.535556,England,5,2.82521e+06
4,Chelsea,London,Stamford Bridge,42449,51.481667,-0.191111,England,5,2.82521e+06


# Importing needed libraries

In [23]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Mapping stadium data

In [25]:
map_stadiums = folium.Map(location=[0, 0], zoom_start=10)

# add markers to map
for Latitude, Longitude, Team, Stadium in zip(result['Latitude'], result['Longitude'], result['Team'], result['Stadium']):
    label = '{}, {}'.format(Team, Stadium)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stadiums)  
    
map_stadiums

# Getting Foursquare data

In [28]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [30]:
LIMIT = 20
radius = 300

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Stadium', 
                  'Stadium Latitude', 
                  'Stadium Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
stadium_venues = getNearbyVenues(names=result['Stadium'],
                                   latitudes=result['Latitude'],
                                   longitudes=result['Longitude'],
                                   radius=radius
                                )

Emirates Stadium 
Villa Park 
Ewood Park 
Reebok Stadium 
Stamford Bridge 
Goodison Park 
Craven Cottage 
Anfield 
Etihad Stadium
Old Trafford 
Sports Direct Arena
Carrow Road 
Loftus Road 
Britannia Stadium 
Stadium of Light 
Liberty Stadium 
White Hart Lane 
The Hawthorns 
DW Stadium 
Molineux Stadium 
Oakwell 
St Andrew's Stadium 
Bloomfield Road 
American Express Community Stadium 
Ashton Gate 
Turf Moor 
Cardiff City Stadium 
Ricoh Arena 
Selhurst Park 
Pride Park Stadium 
Keepmoat Stadium 
KC Stadium 
Portman Road 
Elland Road 
King Power Stadium 
Riverside Stadium 
The Den 
City Ground 
London Road Stadium 
Fratton Park 
Madejski Stadium 
St Mary's Stadium 
Vicarage Road 
Boleyn Ground 
Almondvale Stadium 
Caledonian Stadium 
Celtic Park 
Dens Park 
East End Park 
Easter Road 
Falkirk Stadium 
Fir Park 
Firhill Stadium 
Ibrox Stadium 
McDiarmid Park 
New Douglas Park 
Pittodrie Stadium 
Rugby Park 
St. Mirren Park 
Tannadice Park 
Tynecastle Stadium 
Victoria Park 
SGL arena 
Ba

In [34]:
stadium_venues.shape

(1064, 7)

In [35]:
stadium_venues.head(10)

,Stadium,Stadium Latitude,Stadium Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Emirates Stadium,51.555,-0.108611,Emirates Stadium,51.554296,-0.109030,Soccer Stadium
1,Emirates Stadium,51.555,-0.108611,The Armoury,51.554259,-0.110306,Miscellaneous Shop
2,Emirates Stadium,51.555,-0.108611,Xi'an Impression,51.553858,-0.110306,Shaanxi Restaurant
3,Emirates Stadium,51.555,-0.108611,East Stand,51.554867,-0.107082,Soccer Stadium
4,Emirates Stadium,51.555,-0.108611,All Arsenal (Shop),51.556695,-0.106098,Clothing Store
5,Emirates Stadium,51.555,-0.108611,Ashburton Triangle and Arsenal Museum,51.556403,-0.107927,History Museum
6,Emirates Stadium,51.555,-0.108611,Wolkite Kitfo Restaurant & Cafe,51.556065,-0.111416,Ethiopian Restaurant
7,Emirates Stadium,51.555,-0.108611,Little Wonder Cafe,51.554062,-0.110541,Breakfast Spot
8,Emirates Stadium,51.555,-0.108611,Co-op Food,51.553364,-0.109831,Grocery Store
9,Emirates Stadium,51.555,-0.108611,Domino's Pizza,51.556213,-0.110868,Pizza Place


In [36]:
stadium_venues.groupby('Stadium').count()

,Stadium Latitude,Stadium Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Stadium,,,,,,
Allianz Arena,20,20,20,20,20,20
Allianz Riviera,4,4,4,4,4,4
Almondvale Stadium,3,3,3,3,3,3
American Express Community Stadium,5,5,5,5,5,5
Anfield,16,16,16,16,16,16
Anoeta,14,14,14,14,14,14
Ashton Gate,6,6,6,6,6,6
Balaídos,7,7,7,7,7,7
BayArena,14,14,14,14,14,14


# Extracting venue categories

In [37]:
# one hot encoding
stadium_onehot = pd.get_dummies(stadium_venues[['Venue Category']], prefix="", prefix_sep="")

# add stadium column back to dataframe
stadium_onehot['Stadium'] = stadium_venues['Stadium'] 

# move stadium column to the first column
fixed_columns = [stadium_onehot.columns[-1]] + list(stadium_onehot.columns[:-1])
stadium_onehot = stadium_onehot[fixed_columns]

stadium_onehot.head()

,Women's Store,American Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beer Garden,Bistro,Boarding House,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Farm,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Arena,Hockey Rink,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Light Rail Station,Lighthouse,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,River,Rock Club,Rugby Stadium,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theme Park,Theme Restaurant,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Video Game Store,Warehouse Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,Emirates Stadium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Emirates Stadium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Emirates Stadium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,Emirates Stadium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

# Finding rate of occurence of venues

In [39]:
stadium_grouped = stadium_onehot.groupby('Stadium').mean().reset_index()
stadium_grouped

,Stadium,Women's Store,American Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beer Garden,Bistro,Boarding House,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Farm,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Arena,Hockey Rink,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Light Rail Station,Lighthouse,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,River,Rock Club,Rugby Stadium,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theme Park,Theme Restaurant,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Video Game Store,Warehouse Store
0,Allianz Arena,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.050000,0.000,0.000000,0.000000,0.050000,0.050000,0.000000,0.000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.00,0.000000,0.00,0.250000,0.000000,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.050000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.00,0.00,0.00,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.050000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.00,0.000000
1,Allianz Riviera,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.00,0

# Preparing data for clustering

In [42]:
dataSet = pd.merge(stadium_grouped, 
                 result,
                 on='Stadium', 
                 how='left')

In [45]:
dataSet.head(10)

,Stadium,Women's Store,American Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beer Garden,Bistro,Boarding House,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Farm,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Arena,Hockey Rink,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Light Rail Station,Lighthouse,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,River,Rock Club,Rugby Stadium,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theme Park,Theme Restaurant,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Video Game Store,Warehouse Store,Team,City,Capacity,Latitude,Longitude,Country,Ranking,"US dollars, millions"
0,Allianz Arena,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.05,0.000,0.0,0.0,0.05,0.05,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0500,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.05,0.0,0.0,0.0000,0.0,0.0500,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.200000,0.0000,0.0,0.000,0.050000,0.050000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.05,0.0,0.0,0.0,Bayern Munich,Munich,71000,48.218775,11.624753,Germany,4,3.99676e+06
1,Allianz Riviera,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.00,0.000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0

In [47]:
dataSet_clustering = dataSet.drop('Stadium', 1)

In [48]:
dataSet_clustering = dataSet_clustering.drop('Team', 1)
dataSet_clustering = dataSet_clustering.drop('City', 1)
dataSet_clustering = dataSet_clustering.drop('Latitude', 1)
dataSet_clustering = dataSet_clustering.drop('Longitude', 1)
dataSet_clustering = dataSet_clustering.drop('Country', 1)
dataSet_clustering = dataSet_clustering.drop('Ranking', 1)

In [49]:
dataSet_clustering.head(10)

,Women's Store,American Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beer Garden,Bistro,Boarding House,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Farm,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Arena,Hockey Rink,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Light Rail Station,Lighthouse,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,River,Rock Club,Rugby Stadium,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theme Park,Theme Restaurant,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Video Game Store,Warehouse Store,Capacity,"US dollars, millions"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.05,0.000,0.0,0.0,0.05,0.05,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0500,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.05,0.0,0.0,0.0000,0.0,0.0500,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.200000,0.0000,0.0,0.000,0.050000,0.050000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.05,0.0,0.0,0.0,71000,3.99676e+06
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.00,0.000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.250000,0.0000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.25000

In [50]:
dataSet_clustering = dataSet_clustering.dropna() 

In [51]:
dataSet_clustering.head(10)

,Women's Store,American Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beer Garden,Bistro,Boarding House,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Farm,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Arena,Hockey Rink,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Light Rail Station,Lighthouse,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,River,Rock Club,Rugby Stadium,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theme Park,Theme Restaurant,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Video Game Store,Warehouse Store,Capacity,"US dollars, millions"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.05,0.000,0.0,0.0,0.05,0.05,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0500,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.05,0.0,0.0,0.000000,0.0,0.0500,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.200000,0.0000,0.0,0.000,0.050000,0.050000,0.0,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.05,0.0,0.0,0.0,71000,3.99676e+06
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.00,0.000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.250000,0.0000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.25,0.0,0.00

# Normalizing Capacity and GDP data to (0,1) interval

In [52]:
dataSet_clustering["Capacity"]=((dataSet_clustering["Capacity"]-dataSet_clustering["Capacity"].min())/(dataSet_clustering["Capacity"].max()-dataSet_clustering["Capacity"].min()))*1

In [53]:
dataSet_clustering["US dollars, millions"]=((dataSet_clustering["US dollars, millions"]-dataSet_clustering["US dollars, millions"].min())/(dataSet_clustering["US dollars, millions"].max()-dataSet_clustering["US dollars, millions"].min()))*1

# Clustering the data

In [54]:
# set number of clusters
kclusters = 7

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dataSet_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 3, 2, 0, 3, 0, 5, 0, 3], dtype=int32)

In [57]:
stadium_venues_sorted.shape

(136, 6)

In [58]:
stadium_venues_sorted.head()

,Stadium,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Allianz Arena,Lounge,Soccer Stadium,Restaurant,Café,Sporting Goods Shop
1,Allianz Riviera,Soccer Stadium,Tram Station,Supermarket,French Restaurant,Financial or Legal Service
2,Almondvale Stadium,Soccer Stadium,Gastropub,Supermarket,French Restaurant,Forest
3,American Express Community Stadium,Bar,Sporting Goods Shop,Train Station,Coffee Shop,Soccer Stadium
4,Anfield,Pub,Bakery,Soccer Stadium,Restaurant,Outdoor Sculpture


In [64]:
dataset_stadiums = dataSet.dropna()

In [66]:
column = dataset_stadiums['Stadium']

In [79]:
result3 = pd.merge(column, 
                 stadium_venues_sorted,
                 on='Stadium', 
                 how='left')

# Preparing data for presentation on map

In [80]:
# add clustering labels
result3.insert(0, 'Cluster Labels', kmeans.labels_)

stadium_merged = result

In [81]:
stadium_merged = stadium_merged.join(result3.set_index('Stadium'), on='Stadium')

stadium_merged.head() 

,Team,City,Stadium,Capacity,Latitude,Longitude,Country,Ranking,"US dollars, millions",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Arsenal,London,Emirates Stadium,60361,51.555000,-0.108611,England,5,2.82521e+06,2.0,Soccer Stadium,Grocery Store,Ethiopian Restaurant,Miscellaneous Shop,Soccer Field
1,Aston Villa,Birmingham,Villa Park,42785,52.509167,-1.884722,England,5,2.82521e+06,4.0,Soccer Stadium,Pub,Sporting Goods Shop,Soccer Field,Donut Shop
2,Blackburn Rovers,Blackburn,Ewood Park,31154,53.728611,-2.489167,England,5,2.82521e+06,3.0,Dive Bar,Sporting Goods Shop,Sports Bar,Grocery Store,Soccer Stadium
3,Bolton Wanderers,Bolton,Reebok Stadium,28100,53.580556,-2.535556,England,5,2.82521e+06,3.0,Sporting Goods Shop,Hotel,Chinese Restaurant,Sports Club,Furniture / Home Store
4,Chelsea,London,Stamford Bridge,42449,51.481667,-0.191111,England,5,2.82521e+06,2.0,Soccer Stadium,Gym / Fitness Center,Museum,Café,Soccer Field


In [86]:
stadium_merged2 = stadium_merged.dropna()

In [87]:
 stadium_merged2['Cluster Labels'] =  stadium_merged2['Cluster Labels'].apply(np.int64)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Representation of clusters on the map

In [88]:
map_clusters = folium.Map(location=[0, 0], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stadium_merged2['Latitude'], stadium_merged2['Longitude'], stadium_merged2['Stadium'], stadium_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Exploration of clusters

In [89]:
stadium_merged2.loc[stadium_merged2['Cluster Labels'] == 0, stadium_merged2.columns[[1] + list(range(5, stadium_merged2.shape[1]))]]

,City,Longitude,Country,Ranking,"US dollars, millions",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
85,Almería,-2.435278,Spain,14,1.42619e+06,0,Tapas Restaurant,Soccer Stadium,Warehouse Store,Fried Chicken Joint,Forest
86,Bilbao,-2.950366,Spain,14,1.42619e+06,0,Café,Bar,Metro Station,Restaurant,Snack Place
87,Madrid,-3.720606,Spain,14,1.42619e+06,0,Pub,Bar,Park,Restaurant,Museum
89,Seville,-5.981389,Spain,14,1.42619e+06,0,Breakfast Spot,Soccer Stadium,Restaurant,Gym / Fitness Center,Spanish Restaurant
90,Vigo,-8.739711,Spain,14,1.42619e+06,0,Café,Soccer Stadium,Breakfast Spot,Diner,Cocktail Bar
91,Elche,-0.663333,Spain,14,1.42619e+06,0,Spanish Restaurant,Bar,Tennis Court,Soccer Stadium,Financial or Legal Service
92,Cornellà de Llobregat,2.075667,Spain,14,1.42619e+06,0,Clothing Store,Coffee Shop,Italian Restaurant,Soccer Stadium,Shopping Mall
93,Getafe,-3.714722,Spain,14,1.42619e+06,0,Diner,Restaurant,Plaza,Soccer Stadium,Gym / Fitness Center
94,Granada,-3.595736,Spain,14,1.42619e+06,0,Pizza Place,Spanish Restaurant,Italian Restaurant,Japanese Restaurant,Soccer Stadium
95,Valencia,-0.364167,Spain,14,1.42619e+06,0,Clothing Store,Soccer Stadium,Basketball Stadium,Electronics Store,Coffee Shop


In [90]:
stadium_merged2.loc[stadium_merged2['Cluster Labels'] == 1, stadium_merged2.columns[[1] + list(range(5, stadium_merged2.shape[1]))]]

,City,Longitude,Country,Ranking,"US dollars, millions",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
64,Munich,11.624753,Germany,4,3.99676e+06,1,Lounge,Soccer Stadium,Restaurant,Café,Sporting Goods Shop
65,Dortmund,7.451842,Germany,4,3.99676e+06,1,Soccer Stadium,Museum,Pool,Sporting Goods Shop,Sports Bar
66,Mönchengladbach,6.385464,Germany,4,3.99676e+06,1,Soccer Stadium,Drugstore,Hotel,Restaurant,Football Stadium
71,Hanover,9.731197,Germany,4,3.99676e+06,1,Soccer Stadium,Pool,Sporting Goods Shop,Sports Bar,Financial or Legal Service
72,Berlin,13.239444,Germany,4,3.99676e+06,1,Soccer Stadium,Lounge,Pool,Restaurant,Sporting Goods Shop
73,Sinsheim,8.888281,Germany,4,3.99676e+06,1,Lounge,Souvenir Shop,Soccer Stadium,French Restaurant,Forest
74,Mainz,8.224167,Germany,4,3.99676e+06,1,Soccer Stadium,Sporting Goods Shop,Restaurant,Fast Food Restaurant,Football Stadium
76,Gelsenkirchen,7.067589,Germany,4,3.99676e+06,1,Soccer Stadium,Pub,Historic Site,Museum,Fast Food Restaurant
79,Wolfsburg,10.803889,Germany,4,3.99676e+06,1,Soccer Stadium,Buffet,Restaurant,Financial or Legal Service,Forest
80,Düsseldorf,6.733056,Germany,4,3.99676e+06,1,Soccer Stadium,Basketball Court,Track Stadium,Lounge,Athletics & Sports


In [91]:
stadium_merged2.loc[stadium_merged2['Cluster Labels'] == 2, stadium_merged2.columns[[1] + list(range(5, stadium_merged2.shape[1]))]]

,City,Longitude,Country,Ranking,"US dollars, millions",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,London,-0.108611,England,5,2.82521e+06,2,Soccer Stadium,Grocery Store,Ethiopian Restaurant,Miscellaneous Shop,Soccer Field
4,London,-0.191111,England,5,2.82521e+06,2,Soccer Stadium,Gym / Fitness Center,Museum,Café,Soccer Field
5,Liverpool,-2.966389,England,5,2.82521e+06,2,Sporting Goods Shop,Pub,Outdoor Sculpture,Soccer Stadium,Dry Cleaner
7,Liverpool,-2.960828,England,5,2.82521e+06,2,Pub,Bakery,Soccer Stadium,Restaurant,Outdoor Sculpture
8,Manchester,-2.200292,England,5,2.82521e+06,2,Athletics & Sports,Soccer Stadium,Café,Lounge,Sporting Goods Shop
10,Newcastle upon Tyne,-1.621667,England,5,2.82521e+06,2,Coffee Shop,Soccer Stadium,Pub,Sports Bar,Bar
14,Sunderland,-1.388200,England,5,2.82521e+06,2,Soccer Stadium,Hotel,Gym Pool,Athletics & Sports,Spa
16,London,-0.065833,England,5,2.82521e+06,2,Hostel,Greek Restaurant,Fried Chicken Joint,Soccer Stadium,Sporting Goods Shop
29,Derby,-1.447222,England,5,2.82521e+06,2,Coffee Shop,English Restaurant,Clothing Store,Sandwich Place,Gym / Fitness Center
33,Leeds,-1.572222,England,5,2.82521e+06,2,Pub,Sandwich Place,Bus Stop,Sporting Goods Shop,Fast Food Restaurant


In [92]:
stadium_merged2.loc[stadium_merged2['Cluster Labels'] == 3, stadium_merged2.columns[[1] + list(range(5, stadium_merged2.shape[1]))]]

,City,Longitude,Country,Ranking,"US dollars, millions",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Blackburn,-2.489167,England,5,2.82521e+06,3,Dive Bar,Sporting Goods Shop,Sports Bar,Grocery Store,Soccer Stadium
3,Bolton,-2.535556,England,5,2.82521e+06,3,Sporting Goods Shop,Hotel,Chinese Restaurant,Sports Club,Furniture / Home Store
6,London,-0.221667,England,5,2.82521e+06,3,Bar,Soccer Field,Soccer Stadium,Athletics & Sports,Park
11,Norwich,1.308653,England,5,2.82521e+06,3,Soccer Stadium,Sporting Goods Shop,Supermarket,Restaurant,Coffee Shop
15,Swansea,-3.935100,England,5,2.82521e+06,3,American Restaurant,Pizza Place,English Restaurant,Italian Restaurant,Coffee Shop
18,Wigan,-2.653889,England,5,2.82521e+06,3,Athletics & Sports,Track Stadium,Coffee Shop,Sporting Goods Shop,Gym
19,Wolverhampton,-2.130278,England,5,2.82521e+06,3,Performing Arts Venue,Sporting Goods Shop,Coffee Shop,Sandwich Place,Café
21,Birmingham,-1.868189,England,5,2.82521e+06,3,Discount Store,Hotel,Pharmacy,Convenience Store,Sporting Goods Shop
22,Blackpool,-3.048056,England,5,2.82521e+06,3,Hotel,Pub,Park,Soccer Stadium,Outdoor Sculpture
23,Brighton,-0.083278,England,5,2.82521e+06,3,Bar,Sporting Goods Shop,Train Station,Coffee Shop,Soccer Stadium


In [93]:
stadium_merged2.loc[stadium_merged2['Cluster Labels'] == 4, stadium_merged2.columns[[1] + list(range(5, stadium_merged2.shape[1]))]]

,City,Longitude,Country,Ranking,"US dollars, millions",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Birmingham,-1.884722,England,5,2.82521e+06,4,Soccer Stadium,Pub,Sporting Goods Shop,Soccer Field,Donut Shop
12,London,-0.232222,England,5,2.82521e+06,4,Grocery Store,Soccer Stadium,Fast Food Restaurant,Football Stadium,Food Court
13,Stoke-on-Trent,-2.175556,England,5,2.82521e+06,4,Soccer Stadium,English Restaurant,Gym,Warehouse Store,Financial or Legal Service
17,West Bromwich,-1.963889,England,5,2.82521e+06,4,Soccer Stadium,Soccer Field,Fast Food Restaurant,Financial or Legal Service,Football Stadium
20,Barnsley,-1.467500,England,5,2.82521e+06,4,Soccer Stadium,IT Services,Warehouse Store,Financial or Legal Service,Forest
25,Burnley,-2.230278,England,5,2.82521e+06,4,Bar,Soccer Stadium,Fried Chicken Joint,Forest,Football Stadium
35,Middlesbrough,-1.216944,England,5,2.82521e+06,4,Soccer Stadium,Sports Bar,Sculpture Garden,Fast Food Restaurant,Football Stadium
41,Southampton,-1.391111,England,5,2.82521e+06,4,Sporting Goods Shop,Soccer Stadium,Warehouse Store,Fried Chicken Joint,Forest
108,Ajaccio,8.776725,France,6,2.77754e+06,4,Soccer Stadium,Hotel,Brewery,Soccer Field,Financial or Legal Service
109,Bastia,9.442619,France,6,2.77754e+06,4,Fast Food Restaurant,Soccer Stadium,Financial or Legal Service,Forest,Football Stadium


In [94]:
stadium_merged2.loc[stadium_merged2['Cluster Labels'] == 5, stadium_merged2.columns[[1] + list(range(5, stadium_merged2.shape[1]))]]

,City,Longitude,Country,Ranking,"US dollars, millions",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
62,Augsburg,10.882222,Germany,4,3.99676e+06,5,Soccer Field,Warehouse Store,Financial or Legal Service,Forest,Football Stadium
63,Leverkusen,7.002206,Germany,4,3.99676e+06,5,Soccer Stadium,Lounge,Fast Food Restaurant,Soccer Field,Hotel
67,Braunschweig,10.521667,Germany,4,3.99676e+06,5,Soccer Stadium,Bus Stop,Sports Bar,Gift Shop,Fish & Chips Shop
68,Frankfurt,8.645458,Germany,4,3.99676e+06,5,Soccer Stadium,Sporting Goods Shop,Museum,Hotel,Gym / Fitness Center
69,Freiburg,7.893056,Germany,4,3.99676e+06,5,Soccer Field,Hostel,Gas Station,Soccer Stadium,Pool
70,Hamburg,9.898617,Germany,4,3.99676e+06,5,Outdoor Sculpture,Athletics & Sports,Food,Sporting Goods Shop,Lounge
75,Nuremberg,11.125833,Germany,4,3.99676e+06,5,Pool,Concert Hall,Gastropub,Lounge,Soccer Stadium
77,Stuttgart,9.232031,Germany,4,3.99676e+06,5,Soccer Field,Plaza,Greek Restaurant,Soccer Stadium,Spa
78,Bremen,8.837628,Germany,4,3.99676e+06,5,Soccer Stadium,Museum,Tennis Stadium,Food Court,Sporting Goods Shop
81,Fürth,10.999167,Germany,4,3.99676e+06,5,Soccer Stadium,Pizza Place,Gas Station,Liquor Store,Financial or Legal Service


In [95]:
stadium_merged2.loc[stadium_merged2['Cluster Labels'] == 6, stadium_merged2.columns[[1] + list(range(5, stadium_merged2.shape[1]))]]

,City,Longitude,Country,Ranking,"US dollars, millions",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,Stretford,-2.291389,England,5,2.82521e+06,6,Soccer Stadium,Theme Restaurant,Fast Food Restaurant,Souvenir Shop,Train Station
88,Barcelona,2.122802,Spain,14,1.42619e+06,6,Soccer Stadium,Soccer Field,Museum,Football Stadium,Spanish Restaurant
99,Madrid,-3.688350,Spain,14,1.42619e+06,6,Spanish Restaurant,Soccer Stadium,Museum,Sporting Goods Shop,Cocktail Bar


In [96]:
stadium_merged2.loc[stadium_merged2['Cluster Labels'] == 7, stadium_merged2.columns[[1] + list(range(5, stadium_merged2.shape[1]))]]

,City,Longitude,Country,Ranking,"US dollars, millions",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


# For detailed analysis please refere to the Report! Than you for your attention!